### Data Visualize

In [1]:
import os, glob


In [3]:
IM_VAL = r'/workspace/bdd100k/BDD100k/images/val'
LB_VAL = r'/workspace/bdd100k/BDD100k/labels/val'

imgs = glob.glob(os.path.join(IM_VAL, "**", "*.jpg"), recursive=True)
txts = glob.glob(os.path.join(LB_VAL, "**", "*.txt"), recursive=True)
non_empty = [t for t in txts if os.path.getsize(t) > 0]

print("VAL images:", len(imgs))          # kỳ vọng ~10000
print("VAL label files:", len(txts))     # kỳ vọng ~10000 (hoặc ~số ảnh có bbox)
print("VAL non-empty labels:", len(non_empty))  # PHẢI >0


# Lấy 1 ảnh bất kỳ và kiểm tra file nhãn “soi gương”
sample_img = imgs[0]
rel = os.path.relpath(sample_img, IM_VAL).replace("\\","/")
sample_txt = os.path.join(LB_VAL, os.path.splitext(rel)[0] + ".txt")
print("Sample img:", sample_img)
print("Expect txt:", sample_txt, " exists:", os.path.exists(sample_txt),
      " size:", os.path.getsize(sample_txt) if os.path.exists(sample_txt) else -1)


VAL images: 10000
VAL label files: 10000
VAL non-empty labels: 10000
Sample img: /workspace/bdd100k/BDD100k/images/val/b1c66a42-6f7d68ca.jpg
Expect txt: /workspace/bdd100k/BDD100k/labels/val/b1c66a42-6f7d68ca.txt  exists: True  size: 1324


In [7]:
print("train txt:", len(glob.glob(r"/workspace/bdd100k/BDD100k/labels/train/**/*.txt", recursive=True))) 
print("val   txt:", len(glob.glob(r"/workspace/bdd100k/BDD100k/labels/val/**/*.txt",   recursive=True)))


train txt: 69863
val   txt: 10000


### Training

In [8]:
%pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 42.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 44.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 15.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.7/772.7 KB 20.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 20.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 3.9 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 KB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 16.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
import os, yaml

# === 1) Khai báo đúng 10 lớp (theo mapping bạn đã convert) ===
class_names = [
    'person', 'rider', 'car', 'bus', 'truck',
    'bike', 'motor', 'traffic light', 'traffic sign', 'train'
]

# === 2) Tạo data.yaml đúng với cây thư mục hiện tại ===
ROOT = r"/workspace/bdd100k/BDD100k"   # thư mục chứa images/ và labels/
yaml_file = r"/workspace/bdd100k.yaml"

data = {
    "path": ROOT.replace("\\", "/"),   # dùng / cho chắc ăn trên Windows
    "train": "images/train",
    "val": "images/val",
    "test": "images/test",             # có cũng được, không có YOLO sẽ bỏ qua
    "names": class_names
}

os.makedirs(os.path.dirname(yaml_file), exist_ok=True)
with open(yaml_file, "w", encoding="utf-8") as f:
    yaml.dump(data, f, sort_keys=False, allow_unicode=True)

print(f"YOLO config saved to: {yaml_file}")

# (khuyến nghị) sanity check nhanh để chắc thư mục tồn tại
for p in [ROOT, os.path.join(ROOT, "images/train"), os.path.join(ROOT, "images/val")]:
    print("OK" if os.path.exists(p) else "MISSING", "->", p)


YOLO config saved to: /workspace/bdd100k.yaml
OK -> /workspace/bdd100k/BDD100k
OK -> /workspace/bdd100k/BDD100k/images/train
OK -> /workspace/bdd100k/BDD100k/images/val


In [12]:
from ultralytics import YOLO
import os, yaml, torch

yaml_file = r"/workspace/bdd100k.yaml"
assert os.path.exists(yaml_file), f"Không thấy YAML: {yaml_file}"

WEIGHTS = "yolov8n.pt"  # Giữ v8n cho nhất quán

def main():
    # 1) Kiểm tra nhanh GPU
    device = 0 if torch.cuda.is_available() else "cpu"
    print("CUDA available:", torch.cuda.is_available())

    # 2) Khởi tạo model
    model = YOLO(WEIGHTS)
    model.info()

    # 3) Train (workers=0 để tránh lỗi spawn trên Windows)
    results = model.train(
        data=yaml_file,
        epochs=50,          # train thật
        imgsz=640,          # 4GB nên giữ 640
        batch=64,           # auto; nếu OOM => đặt 2 hoặc 3 ban đầu là -1
        workers=4,          # Windows: 0; Linux/Server: 2-4
        device=device,
        cache=True,        # tránh ăn RAM
        deterministic=False, #True
        close_mosaic=10,
        pretrained=True,
        project=r"/workspace/runs/detect",  # nơi lưu
        name="bdd100k_yolov8n",
        save_period=5,      # lưu checkpoint mỗi 5 epoch
        patience=10         # early-stop nếu không cải thiện
    )

    try:
        print("Save dir:", results.save_dir)
    except Exception:
        try:
            print("Save dir:", model.trainer.save_dir)
        except Exception:
            pass

if __name__ == "__main__":
    main()


CUDA available: True
YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs
Ultralytics 8.3.221 🚀 Python-3.10.12 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24238MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/workspace/bdd100k.yaml, degrees=0.0, deterministic=False, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=bdd100k_yolov8n, nbs=64, nms=Fal